### Predict how many crew members are required for future cruise ships

UC Irvine Machine Learning Repository dataset

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [4]:
df = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
df.describe()

DataFrame[summary: string, Ship_name: string, Cruise_line: string, Age: string, Tonnage: string, passengers: string, length: string, cabins: string, passenger_density: string, crew: string]

In [9]:
for ship in df.head(5):
    print(ship)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)


In [10]:
df.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [11]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol = 'Cruise_line', outputCol = 'cruise_cat')
indexed = indexer.fit(df).transform(df)

In [13]:
print(indexed.head(1))

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]


In [14]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [15]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [17]:
assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol = 'features')

In [18]:
output=assembler.transform(indexed)

In [19]:
output.select('features','crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [20]:
final_data = output.select('features','crew')

In [21]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [23]:
print(train_data.describe().show())
print(test_data.describe().show())

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              110|
|   mean|7.544545454545463|
| stddev|3.804294202430837|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+

None
+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                48|
|   mean| 8.366249999999999|
| stddev|2.6377001789518375|
|    min|               1.8|
|    max|              13.6|
+-------+------------------+

None


In [24]:
from pyspark.ml.regression import LinearRegression

In [25]:
ship_lr = LinearRegression(labelCol='crew')

In [28]:
trained_ship_model= ship_lr.fit(train_data)
ship_result = trained_ship_model.evaluate(test_data)

In [30]:
ship_result.rootMeanSquaredError

0.8903086036796864

In [36]:
ship_result.residuals.show() #Showing residual difference between predicted and actual

+--------------------+
|           residuals|
+--------------------+
|  -1.391020496020218|
| -1.1372718191756714|
| -0.6812659474862066|
| -0.6382911594163669|
| -1.3292531867537587|
| -0.5714710201761033|
| 0.29767835170070356|
|-0.26072111255781216|
|  0.6245565377240059|
| -0.7066580698747078|
| -0.5773767515917605|
|-0.31516284809271866|
| -1.2850166276165176|
| 0.10948052619940007|
| -0.8736598374508056|
|  0.7054409743702639|
|  0.6849129752834084|
| -1.2570048402014997|
| -1.3329989464939906|
| -1.1444568036124405|
+--------------------+
only showing top 20 rows



In [31]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              110|
|   mean|7.544545454545463|
| stddev|3.804294202430837|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [32]:
ship_result.r2

0.8836480510257845

In [33]:
print(ship_result.meanAbsoluteError)
print(ship_result.meanSquaredError)

0.7070438764511128
0.7926494097860729


In [34]:
from pyspark.sql.functions import corr
df.select((corr('crew','passengers'))).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

